In [1]:
import nltk
import re
import string
nltk.download('stopwords')
import numpy as np
from matplotlib import pyplot
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
stopwords=nltk.corpus.stopwords.words("english")
wn=nltk.WordNetLemmatizer()
ps=nltk.PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zafreen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df=pd.read_csv("C:/Users/zafreen/Downloads/spam.csv")

In [4]:
df.head(10)
df=df[["v1","v2"]]
df.head(10)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [5]:
# import string
# def remove_punctuation(text):
#     text="".join([char for char in text if char not in string.punctuation])
#     return text
# df['no_punct_text']=df['v2'].apply(lambda x:remove_punctuation(x))
# import re
# def tokenise(text):
#     text=re.split("\W+",text)
#     return text
# df['tokenised_text']=df['no_punct_text'].apply(lambda x:tokenise(x))
# import nltk
# nltk.download('stopwords')
# stopwords=nltk.corpus.stopwords.words("english")
# def remove_stopwords(text):
#     text=[char for char in text if char not in stopwords]
#     return text
# df['no_stopwords_text']=df['tokenised_text'].apply(lambda x:remove_stopwords(x))
# def lemmetize(text):
#     text=[wn.lemmatize(char) for char in text]
#     return text
# df['lemmetized_text']=df['no_stopwords_text'].apply(lambda x:lemmetize(x))
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# count_vector=CountVectorizer(analyzer=data_cleaning)
# count_vector_=count_vector.fit_transform(df['v2'])
# # print(count_vector_.shape)
# # print(count_vector.get_feature_names())
# dff=pd.DataFrame(count_vector_.toarray())
# dff.columns=count_vector.get_feature_names()

# tfidf_vector=TfidfVectorizer(analyzer=data_cleaning)
# tfidf_vector_=tfidf_vector.fit_transform(df['v2'])
# # print(tfidf_vector_.shape)
# # print(tfidf_vector.get_feature_names())
# df_t=pd.DataFrame(tfidf_vector_.toarray())
# df_t.columns=tfidf_vector.get_feature_names()

# ncount_vector=CountVectorizer(ngram_range=(1,2))
# ncount_vector_=ncount_vector.fit_transform(df['v2'])
# # print(count_vector_.shape)
# # print(count_vector.get_feature_names())
# df_n=pd.DataFrame(ncount_vector_.toarray())
# df_n.columns=ncount_vector.get_feature_names()

# df['text_len']=df['v2'].apply(lambda x:len(x)-x.count(" "))
# def count_punctuation(text):
#     count= sum([1 for char in text if char in string.punctuation])
#     return round(count/(len(text)-text.count(" ")),3)*100
# df['Percent_punctuation']=df['v2'].apply(lambda x:count_punctuation(x))
bins=np.linspace(0,200,40)
pyplot.hist(df[df['v1']=='spam']['text_len'],bins,alpha=0.5,normed=True,label="spam")
pyplot.hist(df[df['v1']=='ham']['text_len'],bins,alpha=0.5,normed=True,label="ham")
pyplot.legend(loc="upper left")
pyplot.show()
for i in [1,2,3,4,5]:
    pyplot.hist((df['Percent_punctuation'])**(1/i),bins=40)
    pyplot.show()
X_features=pd.concat([df['text_len'],df['Percent_punctuation'],pd.DataFrame(tfidf_vector_.toarray())],axis=1)
print(dir(RandomForestClassifier))
print(RandomForestClassifier())


rf=RandomForestClassifier(n_jobs=-1)
k_fold=KFold(n_splits=5)
cross_val_score(rf,X_features,df['v1'],cv=k_fold,scoring="accuracy",n_jobs=-1)

In [35]:
rf=RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs=-1)
rf_model=rf.fit(X_train,Y_train)
rf_model.feature_importances_
sorted((zip(rf_model.feature_importances_,X_train.columns)),reverse=True)[0:10]
y_pred=rf_model.predict(x_test)
precesion,recall,fscore,support=score(y_pred,y_test,pos_label="spam",average="binary")
print("Precesion {} Recall {} Accuracy {}".format(round(precesion,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))

[(0.05828317362129984, 'text_len'),
 (0.03814872847200637, 1799),
 (0.025343591931005514, 7331),
 (0.024629056059604457, 3127),
 (0.022122668626052736, 2027),
 (0.019954013674926736, 7162),
 (0.019812624275958216, 4780),
 (0.017646462512092172, 2167),
 (0.016714404731223126, 1357),
 (0.01609407276835166, 5709)]

In [49]:
def test_(n_est,depth):
    rf=RandomForestClassifier(n_estimators=n_est,max_depth=depth,n_jobs=-1)
    rf_model=rf.fit(X_train,Y_train)
    y_pred=rf_model.predict(x_test)
    precesion,recall,fscore,support=score(y_pred,y_test,pos_label="spam",average="binary")
    print("Estimators {} Max_Depth {} Precesion {} Recall {} Accuracy {}".format(n_est,depth,round(precesion,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))

In [50]:
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        test_(n_est,depth)

Estimators 10 Max_Depth 10 Precesion 0.225 Recall 1.0 Accuracy 0.895
Estimators 10 Max_Depth 20 Precesion 0.623 Recall 1.0 Accuracy 0.949
Estimators 10 Max_Depth 30 Precesion 0.768 Recall 1.0 Accuracy 0.969
Estimators 10 Max_Depth None Precesion 0.841 Recall 0.992 Accuracy 0.978
Estimators 50 Max_Depth 10 Precesion 0.219 Recall 1.0 Accuracy 0.894
Estimators 50 Max_Depth 20 Precesion 0.662 Recall 1.0 Accuracy 0.954
Estimators 50 Max_Depth 30 Precesion 0.795 Recall 1.0 Accuracy 0.972
Estimators 50 Max_Depth None Precesion 0.868 Recall 0.978 Accuracy 0.979
Estimators 100 Max_Depth 10 Precesion 0.278 Recall 1.0 Accuracy 0.902
Estimators 100 Max_Depth 20 Precesion 0.642 Recall 1.0 Accuracy 0.952
Estimators 100 Max_Depth 30 Precesion 0.762 Recall 0.983 Accuracy 0.966
Estimators 100 Max_Depth None Precesion 0.861 Recall 1.0 Accuracy 0.981


In [51]:
from sklearn.model_selection import GridSearchCV

In [59]:
rf=RandomForestClassifier()
params={"n_estimators" : [10,150,300],"max_depth":[30,60,90,None]}
gs=GridSearchCV(rf,params,cv=5)
gs_fit=gs.fit(X_features,df['v1'])
pd.DataFrame(gs_fit.cv_results_).sort_values("mean_test_score",ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,23.035829,0.546989,0.232988,0.004286,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.978475,0.976661,0.966786,0.976661,0.975053,0.004192,1
7,12.275443,0.754319,0.165665,0.020927,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.977578,0.973968,0.967684,0.973968,0.974155,0.003616,2
11,24.816205,0.656574,0.242755,0.007691,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977578,0.978475,0.973968,0.965889,0.974865,0.974155,0.004455,3
6,1.198190,0.031794,0.103059,0.005862,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.978475,0.978475,0.975763,0.964093,0.973070,0.973975,0.005332,4
10,13.586022,0.527471,0.181488,0.015922,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.977578,0.973968,0.967684,0.973070,0.973617,0.003347,5


In [70]:
from sklearn.ensemble import GradientBoostingClassifier
def test_g(n_est,depth,lr):
    rf=GradientBoostingClassifier(n_estimators=n_est,max_depth=depth,learning_rate=lr)
    rf_model=rf.fit(X_train,Y_train)
    y_pred=rf_model.predict(x_test)
    precesion,recall,fscore,support=score(y_pred,y_test,pos_label="spam",average="binary")
    print("Estimators {} Max_Depth {} Learning rate {} Precesion {} Recall {} Accuracy {}".format(n_est,depth,lr,round(precesion,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))


In [71]:
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        for lr in [0.01,0.1,1]:
            test_g(n_est,depth,lr)

C:\Users\zafreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Estimators 10 Max_Depth 10 Learning rate 0.01 Precesion 0.0 Recall 0.0 Accuracy 0.865
Estimators 10 Max_Depth 10 Learning rate 0.1 Precesion 0.795 Recall 0.938 Accuracy 0.965
Estimators 10 Max_Depth 10 Learning rate 1 Precesion 0.848 Recall 0.865 Accuracy 0.961


C:\Users\zafreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Estimators 10 Max_Depth 20 Learning rate 0.01 Precesion 0.0 Recall 0.0 Accuracy 0.865
Estimators 10 Max_Depth 20 Learning rate 0.1 Precesion 0.841 Recall 0.876 Accuracy 0.962
Estimators 10 Max_Depth 20 Learning rate 1 Precesion 0.874 Recall 0.874 Accuracy 0.966


C:\Users\zafreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Estimators 50 Max_Depth 20 Learning rate 0.01 Precesion 0.0 Recall 0.0 Accuracy 0.865
Estimators 50 Max_Depth 20 Learning rate 0.1 Precesion 0.868 Recall 0.879 Accuracy 0.966
Estimators 50 Max_Depth 20 Learning rate 1 Precesion 0.861 Recall 0.897 Accuracy 0.968


C:\Users\zafreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Estimators 50 Max_Depth 30 Learning rate 0.01 Precesion 0.0 Recall 0.0 Accuracy 0.865
Estimators 50 Max_Depth 30 Learning rate 0.1 Precesion 0.854 Recall 0.86 Accuracy 0.961
Estimators 50 Max_Depth 30 Learning rate 1 Precesion 0.887 Recall 0.893 Accuracy 0.97


C:\Users\zafreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Estimators 50 Max_Depth None Learning rate 0.01 Precesion 0.0 Recall 0.0 Accuracy 0.865
Estimators 50 Max_Depth None Learning rate 0.1 Precesion 0.861 Recall 0.85 Accuracy 0.961
Estimators 50 Max_Depth None Learning rate 1 Precesion 0.861 Recall 0.861 Accuracy 0.962
Estimators 100 Max_Depth 10 Learning rate 0.01 Precesion 0.768 Recall 0.959 Accuracy 0.964
Estimators 100 Max_Depth 10 Learning rate 0.1 Precesion 0.861 Recall 0.903 Accuracy 0.969
Estimators 100 Max_Depth 10 Learning rate 1 Precesion 0.828 Recall 0.887 Accuracy 0.962
Estimators 100 Max_Depth 20 Learning rate 0.01 Precesion 0.841 Recall 0.876 Accuracy 0.962
Estimators 100 Max_Depth 20 Learning rate 0.1 Precesion 0.874 Recall 0.892 Accuracy 0.969
Estimators 100 Max_Depth 20 Learning rate 1 Precesion 0.861 Recall 0.949 Accuracy 0.975
Estimators 100 Max_Depth 30 Learning rate 0.01 Precesion 0.848 Recall 0.865 Accuracy 0.961
Estimators 100 Max_Depth 30 Learning rate 0.1 Precesion 0.887 Recall 0.87 Accuracy 0.967
Estimators 100 

In [73]:
import re
import nltk
import time
import string
import numpy as np
import pandas as pd
nltk.download('stopwords')
from matplotlib import pyplot
stopwords=nltk.corpus.stopwords.words("english")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

ps=nltk.PorterStemmer()

df=pd.read_csv("C:/Users/zafreen/Downloads/spam.csv")
df=df[["v1","v2"]]

def count_punctuation(text):
    count= sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")),3)*100

df['text_len']=df['v2'].apply(lambda x:len(x)-x.count(" "))
df['Percent_punctuation']=df['v2'].apply(lambda x:count_punctuation(x))

def data_cleaning(text):
    no_punct_text="".join([char for char in text if char not in string.punctuation])
    tokenized_text=re.split("\W+",no_punct_text)
    no_stopwords_text=[char for char in tokenized_text if char not in stopwords]
    lemmetized_text=[ps.stem(char) for char in no_stopwords_text]
    return lemmetized_text

X_train,x_test,Y_train,y_test=train_test_split(df[['v2','text_len','Percent_punctuation']],df['v1'],test_size=0.2)

tfidf_vector=TfidfVectorizer(analyzer=data_cleaning)
tfidf_vector_fit=tfidf_vector.fit(X_train)

tfidf_train=tfidf_vector_fit.transform(X_train['v2'])
tfidf_test=tfidf_vector_fit.transform(x_test['v2'])
X_train_vect=pd.concat([X_train[['text_len','Percent_punctuation']].reset_index(drop=True),pd.DataFrame(tfidf_train.toarray())],axis=1)
x_test_vect=pd.concat([x_test[['text_len','Percent_punctuation']].reset_index(drop=True),pd.DataFrame(tfidf_test.toarray())],axis=1)


rf=RandomForestClassifier(n_estimators=150,max_depth=None,n_jobs=-1)
start=time.time()
rf_model=rf.fit(X_train_vect,Y_train)
end=time.time()
print("fit_time : {}".format(end-start))
start=time.time()
y_pred=rf_model.predict(x_test_vect)
end=time.time()
print("Prediction_time : {}".format(end-start))
precesion,recall,fscore,support=score(y_pred,y_test,pos_label="spam",average="binary")
print("Precesion {} Recall {} Accuracy {}".format(round(precesion,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))



rf=GradientBoostingClassifier(n_estimators=100,max_depth=11,learning_rate=0.1)
start=time.time()
rf_model=rf.fit(X_train_vect,Y_train)
end=time.time()
print("fit_time : {}".format(end-start))
start=time.time()
y_pred=rf_model.predict(x_test_vect)
end=time.time()
print("Prediction_time : {}".format(end-start))
precesion,recall,fscore,support=score(y_pred,y_test,pos_label="spam",average="binary")
print("Precesion {} Recall {} Accuracy {}".format(round(precesion,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zafreen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


fit_time : 0.28389787673950195
Prediction_time : 0.040717124938964844
Precesion 0.518 Recall 0.571 Accuracy 0.891
fit_time : 1.2963886260986328
Prediction_time : 0.00897979736328125
Precesion 0.532 Recall 0.587 Accuracy 0.895
